# Exponential Margin

In [5]:
from facenet_pytorch import MTCNN
import cv2
from PIL import Image
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchvision import transforms
import numpy as np
import os
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from facenet_pytorch import InceptionResnetV1
from torch import optim
from facenet_pytorch import fixed_image_standardization, training
from torch.utils.data import DataLoader

In [7]:
base_dir = 'crop-mask-face/confidence'
file_list = os.listdir(base_dir)
file_list = [item for item in file_list if '.jpg' in item]

In [8]:
from torchvision.models import resnet50
model= resnet50(pretrained = True, progress = True)
from torch import nn

class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.fc = nn.Linear(2048, 50)
        # self.softmax = torch.nn.Softmax(dim = 1)
    
    def forward(self, x):
        x = self.fc(x)
        return x
model.fc = net()

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.load_state_dict(torch.load('model_parameter_unES.pth'), strict=True)
model.to(device)
model.eval()
score_list = []
for file in file_list:
    src = os.path.join(base_dir,file)
    img = cv2.imread(src)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.transpose(2,1,0)
    img = torch.from_numpy(img)
    # print(img.shape)
    img = transforms.Resize((224,224))(img)
    # print(img.shape)
    img = fixed_image_standardization(img)
    img = img.reshape(1, 3, 224, 224).to(device)
    # print(img.shape)
    with torch.no_grad():
        outputs = model(img)
        ps = torch.exp(outputs)
        _,prediction = torch.max(ps, 1)
        maxexp = np.max(ps.cpu().numpy())
        average = np.average(ps.cpu().numpy())
        score = maxexp/average
        score_list.append(score)

In [10]:
non_target_mark = np.percentile(score_list,85)

In [13]:
class CustomImageDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=ToTensor()):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dira (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        image = cv2.imread(img_name)
        # image = image.view((224,224,3))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # image = cv2.equalizeHist(image)
        image = Image.fromarray(image)
        # print(type(image))
        # # image = np.transpose(image,(1,2,0)) 
        label = self.landmarks_frame.iloc[idx, 1]
        label = np.array(label)
        if self.transform:
            image = self.transform(image)
        return image,label

In [14]:
from facenet_pytorch import fixed_image_standardization, training
preprocessing = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    fixed_image_standardization
])

testset = CustomImageDataset(csv_file = 'test.csv',
                              root_dir='crop-mask-face/test/',
                             transform = preprocessing)
testloader = DataLoader(testset, batch_size=1,
                        shuffle=True, num_workers=0)
right_pred_score = []
wrong_pred_score = []
with torch.no_grad():
    for i, (inputs, classes) in enumerate(testloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model(inputs)
        ps = torch.exp(outputs)
        _,prediction = torch.max(ps, 1)
        maxexp = np.max(ps.cpu().numpy())
        # print(maxexp)
        average = np.average(ps.cpu().numpy())
        score = maxexp/average
        if i < 5:
            print("Prediction : ", prediction, ", Score: ", score)
        if prediction == classes:
            right_pred_score.append(score)
        else:
            wrong_pred_score.append(score)

Prediction :  tensor([3], device='cuda:0') , Score:  50.0
Prediction :  tensor([28], device='cuda:0') , Score:  49.996582
Prediction :  tensor([1], device='cuda:0') , Score:  49.99156
Prediction :  tensor([24], device='cuda:0') , Score:  50.0
Prediction :  tensor([34], device='cuda:0') , Score:  23.298098


In [15]:
target_score = right_pred_score + wrong_pred_score 
target_mark = np.percentile(target_score,15)
margin = target_mark - non_target_mark

In [16]:
margin

3.4264989852905288